# NanoVer ASE Client / Server Example

This notebook demonstrates how to set up a molecular simulation with ASE and combine it with NanoVer to run an interactive molecular dynamics (iMD) simulation. 
We then connect to the simulation from a client, so we can look at the data being produced and visualize it.


**Note**: This example demonstrates running iMD simulations using ASE via NanoVer. For tutorials that demonstrate the different ways to interface ASE with OpenMM via NanoVer, check out the tutorials in this directory prefixed with `ase_openmm`.

## Set up an ASE simulation

First, we set up an ASE simulation. We're going to do a simple lattice of copper particles under an EMT potential, but you can use [any supported calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html#module-ase.calculators)!

In [1]:
from ase import units
from ase.calculators.emt import EMT
from ase.lattice.cubic import FaceCenteredCubic
from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

In [2]:
size = 2
# Set up a crystal
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          symbol="Cu",
                          size=(size, size, size),
                          pbc=True)
# Describe the interatomic interactions with Effective Medium Theory
atoms.calc = EMT()
# Set the atomic momenta to 300 Kelvin. 
MaxwellBoltzmannDistribution(atoms, temperature_K=300)

Next we need to define the component of the molecular dynamics simulation that performs the numerical integration of the equations of motion. We're going to run Langevin dynamics at room temperature:

In [3]:
dyn = Langevin(atoms, timestep=1 * units.fs, temperature_K=300, friction=0.5)

Let's check the dynamics are working by running one simulation step and checking the potential energy.

**WARNING**: when accessing data via the `dyn` object, the values we retrieve are in the *units used internally by ASE*. *These units **differ** from those of NanoVer*, so bear this in mind when examining data this way!

In [4]:
dyn.run(steps = 1)
dyn.get_number_of_steps()

1

In [5]:
# Note that this is the energy in eV, not kJ mol-1 (the standard units of ASE are different to those of OpenMM and NanoVer)
dyn.atoms.get_potential_energy()

0.02585199101165164

Now let's import the `ASESimulation` class: this allows us to pass our ASE molecular dynamics simulation to NanoVer to run an iMD simulation. We can define our `ASESimulation` from the dynamics object we created above, as follows:

In [6]:
from nanover.omni.ase import ASESimulation

ase_sim = ASESimulation.from_ase_dynamics(dyn)

Let's run a couple of steps to check that everything is still working. 

**NOTE**: the commands below still run the ASE MD simulation directly, (i.e. *NOT* using NanoVer and thus not running an iMD simulation), and the data is still being accessed via the dynamics object defined previously (i.e. `ase_sim.dynamics = dyn`, see above). 

In [7]:
ase_sim.dynamics.run(10)
ase_sim.dynamics.nsteps

11

In [8]:
ase_sim.dynamics.atoms.get_potential_energy()

-0.038808466184859114

Great! Everything's working with the `ASESimulation`. Let's turn our attention to setting up the server and running an iMD simulation.

## Set up the NanoVer iMD server

In NanoVer we use the `OmniRunner` class to serve iMD simulations.

In [9]:
from nanover.omni import OmniRunner

We set up an iMD simulation by passing the `ASESimulation` defined above to an `OmniRunner`: we set `port=0` to let the operating system pick a free port for us.

In [10]:
imd_runner = OmniRunner.with_basic_server(ase_sim, port=0, name="ase_nanover_server") #Try using Shift+TAB+TAB to see what you can set up here.

**Note**: Be careful if you run the above cell multiple times without running `imd_runner.close()` (see below), as you will start multiple servers, which may be discovered if you use autoconnect. You can guard against this by swapping the cell above with:

```python
try:
    imd_runner.close()
except NameError: # If the server hasn't been defined yet, there will be an error
    pass
imd_runner = OmniRunner.with_basic_server(ase_sim, port=0)
```

Our server is now running and discoverable! We can see where it's running by running the following cell:

In [11]:
print(f'{imd_runner.app_server.name}: Running at {imd_runner.app_server.address}:{imd_runner.app_server.port}')

ase_nanover_server: Running at [::]:55508


The `[::]` means it is running on all available addresses, e.g. your WiFi and cabled access, and it's found an available port

Now, let's start the ASE simulation

In [12]:
imd_runner.next()

In [13]:
# print the time to check dynamics is running
print(f'Simulation time: {ase_sim.dynamics.get_time()}, ({ase_sim.dynamics.nsteps} steps)')

Simulation time: 4.61666655057811, (47 steps)


Ok, it's working, so let's leave it running dynamics in a background thread. This is what happens by default when you call `.next()`

That's it, your simulation is running and interactive! If you connect to it from iMD-VR, you'll see something like:

![NanoVer ASE EMT](./images/nanover_ase_emt.png)

You can also visualize it live from within a Jupyter notebook, (check out this [example with NGLView](../basics/nanover_nglview.ipynb)).

## Start an IMD client

Let's have a look at the data that's being produced here, by connecting a python client to the server running the simulation.

In [14]:
from nanover.app import NanoverImdClient
client = NanoverImdClient.connect_to_single_server(port=imd_runner.app_server.port)

Subscribe to the stream of frames. By default the server will try to send 30 frames per seconds—if it produces more frames, some frames will be skipped to maintain the cadence.

In [15]:
client.subscribe_to_frames()

Let's grab one of the frames and examine some of its values

In [16]:
client.wait_until_first_frame()
frame = client.current_frame

In [17]:
frame.particle_count

32

In [18]:
# Print the potential energy, here in kJ mol-1 as we are accessing it via NanoVer
frame.potential_energy # use TAB to see what's available!

119.54828003152164

See here that as we have accessed the potential energy via the frame output by NanoVer, so it is given in NanoVer's standard unit of energy, kJ/mol.

We can also print out the raw underlying data that is sent from the server to a client, and see all the simulation data that is being transmitted:

In [19]:
# view the latest frame
client.current_frame.raw

values {
  key: "system.simulation.counter"
  value {
    number_value: 0
  }
}
values {
  key: "server.timestamp"
  value {
    number_value: 142445.64141025
  }
}
values {
  key: "residue.count"
  value {
    number_value: 1
  }
}
values {
  key: "particle.count"
  value {
    number_value: 32
  }
}
values {
  key: "energy.potential"
  value {
    number_value: 135.45548898665891
  }
}
values {
  key: "energy.kinetic"
  value {
    number_value: 105.77865715380986
  }
}
values {
  key: "chain.count"
  value {
    number_value: 1
  }
}
arrays {
  key: "system.box.vectors"
  value {
    float_values {
      values: 0.722
      values: 0
      values: 0
      values: 0
      values: 0.722
      values: 0
      values: 0
      values: 0
      values: 0.722
    }
  }
}
arrays {
  key: "residue.names"
  value {
    string_values {
      values: "ASE"
    }
  }
}
arrays {
  key: "residue.ids"
  value {
    string_values {
      values: "1"
    }
  }
}
arrays {
  key: "residue.chains"
  valu

Now, let's visualize the frame directly with ASE. First we convert the frame back into ASE atoms, and view it. The first frame received by a client always has the topology information

In [20]:
from nanover.ase.converter import frame_data_to_ase
atoms = frame_data_to_ase(client.first_frame, topology=True, positions=False)

We can run this to update the latest positions from the frame (note that NanoVer uses **nanometers** for positions ([the same as OpenMM](http://docs.openmm.org/6.2.0/userguide/theory.html#units)), so we need to convert back to Angstrom to visualise the system using ASE.)

In [21]:
import numpy as np
# set the positions to match the latest frame data
atoms.set_positions(np.array(client.latest_frame.particle_positions) * 10)

ASE has a cool little 2D visualizer

In [22]:
from ase.visualize import view
view(atoms)

<Popen: returncode: None args: ['/opt/homebrew/Caskroom/miniforge/base/envs/...>

And a 3D visualizer! (Check your browser)

In [23]:
view(atoms, viewer='x3d')

# Gracefully terminate!!

It is very important to close your servers, otherwise they'll get in the way by blocking ports, and your clients may autoconnect to the wrong server! We close the server calling `.close()`

In [24]:
imd_runner.close()

Note that outside of a notebook, you can use `with` statements to let NanoVer clean up after itself:

In [25]:
with OmniRunner.with_basic_server(ASESimulation.from_ase_dynamics(dyn), port=0) as imd_runner:
    imd_runner.next()

# Next Steps

This notebook demonstrated how to use NanoVer to run iMD simulations using ASE for a toy system. There's plenty more that can be done!

* Run bigger molecular mechanics simulations using OpenMM. The [nanotube](./ase_openmm_nanotube.ipynb) and [neuraminidase](./ase_openmm_neuraminidase.ipynb) examples show how to set up simulations that combine OpenMM with ASE.
* Use the functionality of ASE change the parameters of an OpenMM simulation on the fly. See the [graphene](./ase_openmm_graphene.ipynb) example.
* To understand what's going on under the hood, check out the [NanoVer frame explained](../fundamentals/frame.ipynb) example. 